In [110]:
from logger import logger
from fastapi import FastAPI
import uvicorn
from PIL import Image
import torch
import numpy as np
from numpy.linalg import norm
# Load model directly
import timm
from transformers import AutoImageProcessor, AutoModelForImageClassification, AutoModel, SwinModel, SwinConfig, \
    AutoConfig, ViTImageProcessor, ViTFeatureExtractor
import config as cnf

from src.routes.routes import router


app = FastAPI()
app.include_router(router)

In [119]:
logger.info("Fast API application running on port: %s and host: %s", cnf.port, cnf.host)
# uvicorn.run(app='main:app', port=int(cnf.port),host=cnf.host)
processor1 = AutoImageProcessor.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
model1 = AutoModelForImageClassification.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
torch.save(model1.state_dict(), f"routes/swin1.pt")

2024-09-16 23:27:49,322 pytesting [INFO] 1 - <module>: Fast API application running on port: 8001 and host: localhost
2024-09-16 23:27:49,333 urllib3.connectionpool [DEBUG] 291 - _get_conn: Resetting dropped connection: huggingface.co
2024-09-16 23:27:50,086 urllib3.connectionpool [DEBUG] 546 - _make_request: https://huggingface.co:443 "HEAD /microsoft/swin-tiny-patch4-window7-224/resolve/main/preprocessor_config.json HTTP/11" 200 0
2024-09-16 23:27:50,483 urllib3.connectionpool [DEBUG] 546 - _make_request: https://huggingface.co:443 "HEAD /microsoft/swin-tiny-patch4-window7-224/resolve/main/config.json HTTP/11" 200 0


In [112]:
img = Image.open('/home/rahul/Documents/pytest-learning/signature (1).jpg')

In [113]:
inputs = processor1(images=img, return_tensors="pt")
outputs = model1(**inputs)
A = outputs.logits[0].tolist()
print(len(outputs.logits[0].tolist()))

1000


In [114]:
# make streams
from io import BytesIO
import json
with open('routes/config.json', "rb") as fh:
    mc = json.load(fh)
mc
with open('routes/preprocessor_config.json', "rb") as fh:
    pc = json.load(fh)
  "feature_extractor_type": "ViTFeatureExtractor",
  "image_mean": [
pc
# processor = AutoImageProcessor.from_pretrained('microsoft/swin-tiny-patch4-window7-224')

{'do_normalize': True,
 'do_resize': True,
 'feature_extractor_type': 'ViTFeatureExtractor',
 'image_mean': [0.485, 0.456, 0.406],
 'image_std': [0.229, 0.224, 0.225],
 'resample': 3,
 'size': 224}

In [115]:
processor = ViTFeatureExtractor(
    do_resize=pc["do_resize"],
    size=pc["size"],
    do_normalize=pc["do_normalize"],
    image_mean=pc["image_mean"],
    image_std=pc["image_std"]
)


In [116]:
# processor = AutoImageProcessor.from_pretrained("routes/preprocessor_config.json")
# processor = AutoImageProcessor.from_dict(pc)
config = SwinConfig.from_dict(mc)
# config = AutoConfig.from_pretrained('routes/config.json')
# print(config)
# config = SwinConfig(image_size=sizes[0], depths=depths[0],
#                     embed_dim=embed_dims[0], num_heads=num_heads[0],
#                     window_size=window_sizes[0])
# model = SwinModel(config)
model = AutoModelForImageClassification.from_config(config)
# print(model)
#
model.load_state_dict(torch.load(open('routes/swin.pt', 'rb')))

/tmp/ipykernel_11426/872923281.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(open('routes/swin.pt', 'rb')))


<All keys matched successfully>

In [117]:

# processor = AutoImageProcessor.from_pretrained("routes/preprocessor_config.json")
# model = AutoModel.from_pretrained("routes/config.json")

# logger.info("original model")
# # p = outputs.pooler_output.tolist()
# print(outputs)
# # print(len(p[0]))
# logits = outputs.logits
# sizes = [224, 224, 224, 384]
# num_heads = [[3, 6, 12, 24]]
# depths = [[2, 2, 6, 2]]
# embed_dims = [96]
# window_sizes = [7]

# img = Image.open('/home/rahul/Documents/pytest-learning/signature (1).jpg')
inputs = processor(images=img, return_tensors="pt")
outputs = model(**inputs)
B = outputs.logits[0].tolist()
# print(outputs.logits)
print(len(outputs.logits[0].tolist()))
# # config = SwinConfig.from_pretrained('microsoft/swin-tiny-patch4-window7-224')
# # model = SwinModel(config)
# # print(len(logits[0]))
# # model = timm.create_model('swin_tiny_patch4_window7_224', pretrained=False)
# # # model = torch.load('routes/pytorch_model.bin', weights_only=False)
# # model = model.load_state_dict(torch.load('routes/FE-swin.pt'))
# print(model.eval())
# model = AutoModel.from_pretrained('/home/rahul/Downloads/tf_model.h5')
# model.save_pretrained('routes/', safe_serialization=False)





1000


In [118]:
cosine = np.dot(A, B) / (norm(A) * norm(B))
print("Cosine Similarity:", cosine)

Cosine Similarity: 0.69686597779347
